In [ ]:
import os
import sys
if not "venv" in sys.executable:
    raise EnvironmentError("The script must be run in a virtual environment with ipykernel installed")

user_input = input("(y/n) Would you like to uninstall all existing packages and install the necessary requirements from startingreqs.txt?")

if user_input.lower() == "y":
    #saves current packages to uninstall.txt
    !pip3 freeze > uninstall.txt
    #uninstalls all non-essential packages in environment
    !pip3 uninstall -r uninstall.txt -y
    #installs necessary packages
    !pip3 install -r additionalreqs.txt

import pprint
from transformers import MarianMTModel, MarianTokenizer
import nltk

In [2]:
#gets path of first audio file in podcast folder, excludes .ds_store
podcastfolder = "./Podcasts"
podcastfiles = []
for file in os.listdir(podcastfolder):
    if file != ".DS_Store":
        podcastfiles.append(file)
podcastaudio = podcastfolder + "/" + podcastfiles[0]
podcastaudio

'./Podcasts/podcastsegmentfr.mp3'

In [3]:
test_baseline = input("Run the baseline model for comparison? Will need to install ffmpeg (https://ffmpeg.org/) (y/n)")
if test_baseline.lower() == "y":
    import whisper
    baseline_model_name = "turbo"
    baseline_model = whisper.load_model(baseline_model_name, device="cpu")

In [4]:
if test_baseline.lower() == "y":
    result = baseline_model.transcribe(podcastaudio, beam_size=5, language="fr")
    transcript = result["text"]
    pprint.pprint(transcript, width=150)

/Users/michael.781/Desktop/DataScience/venvMT/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


(" sa capacité à fédérer, c'est-à-dire à faire en sorte que la population se réunisse et soit heureuse ensemble. La France a aussi pu montrer, grâce "
 'aux Jeux Olympiques, sa capacité à construire des infrastructures, donc des bâtiments de qualité, à accueillir le tourisme et à organiser en '
 "général un des événements les plus importants du monde. Voilà, donc ça c'est pour la partie un peu logistique. Et maintenant, si on s'intéresse "
 'plus particulièrement aux performances sportives, la France avait un objectif assez clair pour ces Jeux Olympiques. On voulait faire partie du top '
 '5 des nations au classement des médailles. Autrement dit, être dans les 5 premiers pays en ce qui concerne le nombre de médailles et plus '
 "particulièrement les médailles d'or. Et Cocorico, on a réussi, puisque la France a remporté 64.")


In [ ]:
#CHANGE FOLLOWING TO TAKE TRANSCRIPT FROM TRANSCRIBE FILE
nltk.download('punkt_tab')
sentences = nltk.tokenize.sent_tokenize(transcript_from_other, language='french')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/michael.781/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translation = []
for sentence in sentences:
    input = tokenizer(sentence, return_tensors="pt", padding=True)
    output = model.generate(**input)
    translation.append([tokenizer.decode(t, skip_special_tokens=True) for t in output])


/Users/michael.781/Desktop/DataScience/venvMT/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# mt_model_name = "Helsinki-NLP/opus-mt-fr-en"
# tokenizer = MarianTokenizer.from_pretrained(mt_model_name)
# mt_model = MarianMTModel.from_pretrained(mt_model_name)


# transcript_sentences = nltk.sent_tokenize(transcript, language="french")
# tokenized_sentences = []
# for sentence in transcript_sentences:
#     tokenized_sentence = tokenizer(sentence, return_tensors="pt")
#     tokenized_sentences.append(tokenized_sentence)

# translation = []
# for sentence in tokenized_sentences:
#     input = {"input_ids": sentence["input_ids"][0].unsqueeze(0)}
#     output = mt_model.generate(**input)
#     translation.append(tokenizer.decode(output[0], skip_special_tokens=True))

/Users/michael.781/Desktop/DataScience/venvMT/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
for sentence in translation:
    print(sentence)

its ability to unite, that is to say, to ensure that the population gathers and is happy together.
France was also able to demonstrate, thanks to the Olympic Games, its capacity to build infrastructure, thus quality buildings, to host tourism and to organize in general one of the most important events in the world.
That's it, so that's for the logistical part.
And now, if we are particularly interested in sports performances, France had a clear objective for these Olympic Games.
We wanted to be one of the top five nations in the medal rankings.
In other words, to be in the top 5 countries with regard to the number of medals and more particularly gold medals.
And Cocorico, we made it, since France won 64.
